<a href="https://colab.research.google.com/github/jvishnuvardhan/Stackoverflow_Questions/blob/master/Sequential_set_weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.1

     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 450kB 45.6MB/s 
     |████████████████████████████████| 3.9MB 38.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=90b7c1c39ac4e9f24f50ef948c8c5d8a8543ddd307bf8bb2c939aede7ca30cdf
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installatio

In [23]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28),name='flatten'),
    tf.keras.layers.Dense(128, activation='relu',name='dense1'),
    tf.keras.layers.Dropout(0.2,name = 'dropout'),
    tf.keras.layers.Dense(10, name='dense2')
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #'sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

# create model architecture and compile
model = create_model()
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 84us/sample - loss: 0.2929 - accuracy: 0.9151
Epoch 2/5
60000/60000 [==============================] - 5s 80us/sample - loss: 0.1407 - accuracy: 0.9586
Epoch 3/5
60000/60000 [==============================] - 5s 81us/sample - loss: 0.1046 - accuracy: 0.9687
Epoch 4/5
60000/60000 [==============================] - 5s 81us/sample - loss: 0.0877 - accuracy: 0.9732
Epoch 5/5
10000/10000 [==============================] - 1s 51us/sample - loss: 0.0749 - accuracy: 0.9756


[0.0749235017108731, 0.9756]

In [34]:
#saving weights
model.save_weights('./model_weights', save_format='tf')

#layer weights from the model 
layer_dict = dict([(layer.name, layer) for layer in model.layers])
print(layer_dict)

{'flatten': <tensorflow.python.keras.layers.core.Flatten object at 0x7f42912785c0>, 'dense1': <tensorflow.python.keras.layers.core.Dense object at 0x7f42a099e9e8>, 'dropout': <tensorflow.python.keras.layers.core.Dropout object at 0x7f42a099e400>, 'dense2': <tensorflow.python.keras.layers.core.Dense object at 0x7f42a099e828>}


In [35]:
# creating the same model architecture and compile
loaded_model = create_model()

# This initializes the variables used by the optimizers,
# as well as any stateful metric variables
loaded_model.train_on_batch(x_train[:1], y_train[:1])

[1.6890267, 1.0]

In [36]:
# loading the weights from base_model
for layer in loaded_model.layers:
  layer_name = layer.name
  print(layer.name)
  layer.set_weights(layer_dict[layer_name].get_weights())

flatten
dense1
dropout
dense2


In [37]:
layer_dict[layer_name].get_weights()

[array([[-0.47275245,  0.10067461, -0.08436214, ...,  0.04099992,
          0.18061247,  0.08973061],
        [-0.00537717, -0.32434094, -0.37918666, ..., -0.00488442,
          0.11360058, -0.2372438 ],
        [-0.47799674, -0.26039222,  0.27706173, ..., -0.51480275,
         -0.4688644 ,  0.05977516],
        ...,
        [-0.53242844,  0.00213692,  0.12406264, ...,  0.03461639,
          0.26692173,  0.00618991],
        [ 0.15864497, -0.41063488, -0.09073453, ..., -0.0584425 ,
          0.20724958, -0.04834089],
        [-0.43112192,  0.3208452 ,  0.39599177, ...,  0.21944313,
         -0.58568865, -0.35774255]], dtype=float32),
 array([-0.1027576 ,  0.0158944 , -0.04762117, -0.14266469,  0.08217667,
        -0.01077904, -0.07067968, -0.04857181,  0.2264693 ,  0.0198027 ],
       dtype=float32)]

In [38]:
# check the evaluation before and after are same
loaded_model.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 53us/sample - loss: 0.0749 - accuracy: 0.9756


[0.0749235017108731, 0.9756]

In [28]:
loaded_model.weights

[<tf.Variable 'dense1_5/kernel:0' shape=(784, 128) dtype=float32, numpy=
 array([[-0.07497767,  0.06643964,  0.0383826 , ...,  0.06389119,
          0.05509873, -0.0433367 ],
        [ 0.02283544,  0.07181709,  0.06484554, ...,  0.04714587,
         -0.01553886, -0.03683612],
        [ 0.06168727, -0.01946748, -0.0749439 , ...,  0.07631098,
         -0.06972053,  0.06935648],
        ...,
        [-0.08016176,  0.05188269, -0.07014988, ...,  0.04606266,
         -0.07608803, -0.03082098],
        [-0.02935277,  0.02753586, -0.02420852, ...,  0.00702955,
          0.00192459, -0.07725525],
        [ 0.03336584,  0.07279281, -0.06052596, ..., -0.06546991,
         -0.05761086,  0.00590884]], dtype=float32)>,
 <tf.Variable 'dense1_5/bias:0' shape=(128,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [29]:
model.weights

[<tf.Variable 'dense1_4/kernel:0' shape=(784, 128) dtype=float32, numpy=
 array([[-0.0780191 ,  0.05300913, -0.07283192, ..., -0.05428978,
         -0.03865045, -0.02055129],
        [ 0.07428133, -0.04044281, -0.06939758, ..., -0.03206056,
         -0.05746911, -0.03894283],
        [ 0.01988082,  0.06437551, -0.05596313, ...,  0.03458891,
          0.02757821,  0.0054944 ],
        ...,
        [ 0.02300761, -0.04811935,  0.02948265, ..., -0.07465199,
          0.03527041,  0.01683003],
        [ 0.0167757 ,  0.06856143, -0.04437774, ..., -0.04517156,
          0.01192071, -0.03033222],
        [ 0.02968704,  0.0810136 ,  0.06660468, ..., -0.00855275,
          0.04834392,  0.04045607]], dtype=float32)>,
 <tf.Variable 'dense1_4/bias:0' shape=(128,) dtype=float32, numpy=
 array([ 0.01334221, -0.01322941,  0.18576555, -0.02684958, -0.02712558,
         0.13721676,  0.0083613 , -0.14233507, -0.10852817,  0.00946134,
        -0.012976  ,  0.04805706,  0.14982802,  0.09004564, -0.06731404